# ML match for LOFAR and the combined PanSTARRS WISE catalogue: Compute the $Q_0$

## Configuration

### Load libraries and setup

In [1]:
import numpy as np
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord, search_around_sky
from IPython.display import clear_output
from pymoc import MOC

In [2]:
from mltier import Field, Q_0, parallel_process, describe
from test import gen_rand_cat_inMOC

In [3]:
%load_ext autoreload

In [4]:
%autoreload

In [5]:
from IPython.display import clear_output

In [6]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Area limits

In [7]:
# Busy week Hatfield 2017
ra_down = 170.
ra_up = 190.
dec_down = 46.8
dec_up = 55.9

# Lockman Swire deep region south
ra_down = 161.9
ra_up = 164.3
dec_down = 56.9
dec_up = 58.1

#load the moc that describes the area where the maximum likelihood matching will occur
#MOC_loc = '/lustre/scratch/astro/im281/data/Lockman_swire_deep/dmu13-lockman-hole-1_MOC.fits'
MOC_loc = '/lustre/scratch/astro/im281/data/Lockman_swire_deep/lockman_deep_south_MOC.fits'
MHz150_MOC = MOC(filename=MOC_loc)

In [8]:
field = Field(ra_down, ra_up, dec_down, dec_up, MHz150_MOC)

field is using a MOC
8242912.713030173


## Load data

In [9]:
lofar_150mhz = Table.read('/lustre/scratch/astro/im281/data/LOFAR/deep-lockman-v0.1.cat.fits')

In [10]:
len(lofar_150mhz)

37165

In [11]:
irac_data = Table.read('/lustre/scratch/astro/im281/data/Lockman_swire_deep/SERVS-I1_20160527.fits.gz')

In [12]:
len(irac_data)
np.array(irac_data.colnames)

array(['field', 'name', 'id', 'ra', 'dec', 'flux_aper_1', 'flux_aper_2',
       'flux_aper_3', 'flux_aper_4', 'flux_aper_5', 'fluxerr_aper_1',
       'fluxerr_aper_2', 'fluxerr_aper_3', 'fluxerr_aper_4',
       'fluxerr_aper_5', 'snr_1', 'snr_2', 'snr_3', 'snr_4', 'snr_5',
       'flux_iso', 'flux_auto', 'fluxerr_iso', 'fluxerr_auto', 'snr_iso',
       'snr_auto', 'kron_radius', 'class_star', 'flags', 'mask', 'cov',
       'csnr', 'poly', 'rel'], 
      dtype='<U14')

In [13]:
#load in the matching data either r band or irac

In [14]:
#combined = Table.read("pw.fits")

In [15]:
#lofar_all = Table.read("data/LOFAR_HBA_T1_DR1_catalog_v0.9.srl.fits")

In [16]:
#np.array(combined.colnames)

In [17]:
#np.array(lofar_all.colnames)

### Filter catalogues

In [18]:
lofar = field.filter_catalogue(lofar_150mhz, colnames=('RA','DEC'))

filtering in a MOC


In [19]:
len(lofar)

1409

In [20]:
combined = field.filter_catalogue(irac_data, colnames=('ra','dec'))

filtering in a MOC


In [21]:
len(combined)

59809

In [22]:
#filter r band or irac catalogue with moc

In [23]:
#lofar = field.filter_catalogue(lofar_all, colnames=("RA", "DEC"))

### Sky coordinates

In [24]:
coords_combined = SkyCoord(combined['ra'], 
                           combined['dec'], 
                           unit=(u.deg, u.deg), 
                           frame='icrs')

In [25]:
coords_lofar = SkyCoord(lofar['RA'], 
                       lofar['DEC'], 
                       unit=(u.deg, u.deg), 
                       frame='icrs')

In [26]:
#create field again around the filtered sources to make random source generate quicker
ra_down = min(lofar['RA']) - 10/3600
print(ra_down)
ra_up = max(lofar['RA']) + 10/3600
print(ra_up)
dec_down = min(lofar['DEC']) - 10/3600
print(dec_down)
dec_up = max(lofar['DEC']) + 10/3600
print(dec_up)

#field = Field(ra_down, ra_up, dec_down, dec_up,MHz150_MOC)
field = Field(ra_down, ra_up, dec_down, dec_up, MHz150_MOC)

162.307696761
164.027831073
57.0327409283
57.9992060428
field is using a MOC
8242912.713030173


### Summary of galaxy types in the combined catalogue

In [27]:
#combined_matched = (~np.isnan(combined["i"]) & 
#                    ~np.isnan(combined["W1mag"]))
#np.sum(combined_matched) # Matches

In [28]:
#combined_panstarrs = (~np.isnan(combined["i"]) & 
#                      np.isnan(combined["W1mag"]))
#np.sum(combined_panstarrs) # Only PanSTARSS

In [29]:
#combined_wise =(np.isnan(combined["i"]) & 
#                ~np.isnan(combined["W1mag"]))
#np.sum(combined_wise) # Only WISE

In [30]:
#combined_i = combined_matched | combined_panstarrs
#combined_w1 = combined_matched | combined_wise

## $Q_0$ dependence on the radius

We will iterate 10 times for each radius. However, the error is so small that a direct computation can be accurate to 4 significant figures.

In [31]:
n_iter = 10

In [35]:
rads = list(range(1,26))
rads = np.linspace(0.25,5,26)

### i-band

In [36]:
#q_0_comp_i = Q_0(coords_lofar, coords_combined[combined_i], field)
q_0_comp_i = Q_0(coords_lofar, coords_combined, field)

In [37]:
q_0_rad_i = []
for radius in rads:
    q_0_rad_aux = []
    for i in range(n_iter):
        out = q_0_comp_i(radius=radius)
        q_0_rad_aux.append(out)
    q_0_rad_i.append(np.mean(q_0_rad_aux))
    print("{:7.5f} {:7.5f} +/- {:7.5f} [{:7.5f} {:7.5f}]".format(radius, 
            np.mean(q_0_rad_aux), np.std(q_0_rad_aux), 
            np.min(q_0_rad_aux), np.max(q_0_rad_aux)))

0.25000 0.16240 +/- 0.00066 [0.16097 0.16335]
0.44000 0.35477 +/- 0.00117 [0.35265 0.35679]
0.63000 0.51828 +/- 0.00162 [0.51513 0.52100]
0.82000 0.62965 +/- 0.00115 [0.62816 0.63188]
1.01000 0.70579 +/- 0.00094 [0.70438 0.70779]
1.20000 0.75870 +/- 0.00107 [0.75702 0.76073]
1.39000 0.78260 +/- 0.00139 [0.78052 0.78519]
1.58000 0.81140 +/- 0.00131 [0.80898 0.81338]
1.77000 0.82137 +/- 0.00105 [0.81958 0.82326]
1.96000 0.83671 +/- 0.00130 [0.83399 0.83844]
2.15000 0.84438 +/- 0.00108 [0.84320 0.84723]
2.34000 0.84884 +/- 0.00163 [0.84691 0.85156]
2.53000 0.85470 +/- 0.00125 [0.85260 0.85679]
2.72000 0.86194 +/- 0.00113 [0.86009 0.86473]
2.91000 0.87120 +/- 0.00142 [0.86868 0.87340]
3.10000 0.87332 +/- 0.00148 [0.87067 0.87544]
3.29000 0.87377 +/- 0.00203 [0.87063 0.87724]
3.48000 0.88006 +/- 0.00165 [0.87731 0.88333]
3.67000 0.87749 +/- 0.00146 [0.87475 0.87914]
3.86000 0.87759 +/- 0.00113 [0.87604 0.87968]
4.05000 0.87605 +/- 0.00179 [0.87269 0.87813]
4.24000 0.87402 +/- 0.00252 [0.869

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)
plot(rads, q_0_rad_i)
xlabel("Radius (arcsecs)")
ylabel("$Q_0 i-band$")
ylim([-2, 1]);

### W1-band

In [ ]:
#q_0_comp_w1 = Q_0(coords_lofar, coords_combined[combined_w1], field)

In [ ]:
#q_0_rad_w1 = []
#for radius in rads:
#    q_0_rad_aux = []
#    for i in range(n_iter):
#        out = q_0_comp_w1(radius=radius)
#        q_0_rad_aux.append(out)
#    q_0_rad_w1.append(np.mean(q_0_rad_aux))
#    print("{:2d} {:7.5f} +/- {:7.5f} [{:7.5f} {:7.5f}]".format(radius, 
#            np.mean(q_0_rad_aux), np.std(q_0_rad_aux), 
#            np.min(q_0_rad_aux), np.max(q_0_rad_aux)))

In [ ]:
#plt.rcParams["figure.figsize"] = (5,5)
#plot(rads, q_0_rad_w1)
#xlabel("Radius (arcsecs)")
#ylabel("$Q_0 W1-band$")
#ylim([0, 1]);